# ТЕСТЫ

## тест1

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/illiteracy.txt', sep = '\t', header = 0)

In [3]:
data.head()

,Country,Illit,Births
0,Albania,20.5,1.78
1,Algeria,39.1,2.44
2,Bahrain,15.0,2.34
3,Belize,5.9,2.97
4,Benin,73.5,5.60


In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 3 columns):
Country    94 non-null object
Illit      94 non-null float64
Births     94 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.3+ KB


In [5]:
data.corr()

,Illit,Births
Illit,1.000000,0.768663
Births,0.768663,1.000000


In [6]:
data.corr(method='spearman')

,Illit,Births
Illit,1.000000,0.752962
Births,0.752962,1.000000


## тест2

In [7]:
data_t2 = pd.read_csv('data/water.txt', sep = '\t', header = 0)

In [8]:
data_t2.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [9]:
data_t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
location     61 non-null object
town         61 non-null object
mortality    61 non-null int64
hardness     61 non-null int64
dtypes: int64(2), object(2)
memory usage: 2.0+ KB


### корреляция Пирсона

In [10]:
data_t2.corr()

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


### корреляция Спирмена

In [12]:
data_t2.corr(method='spearman')

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


### корреляция Пирсона

In [53]:
data_t2[data_t2['location'] == 'South'].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [18]:
data_t2[data_t2['location'] == 'North'].corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


### корреляция Мэтьюса

In [86]:
# [[203, 239],
# [718, 515]]
corr_Matt = (239*718 - 515*203) / np.sqrt((239+203) * (239+515) * (515+718) * (203+719))
corr_Matt

0.10894324663311054

корреляция Мэтьюса и уроверь значимости (pv) через статистику хи-квадрат

In [32]:
data_t3 = pd.DataFrame({'male':[239, 515],
                        'female':[203, 718]})

In [33]:
data_t3

,female,male
0,203,239
1,718,515


In [89]:
chi2, pv, dof, expected = chi2_contingency(data_t3)
chi2_contingency(data_t3)

(19.407530788543038,
 1.0558987006638725e-05,
 1L,
 array([[ 243.03402985,  198.96597015],
        [ 677.96597015,  555.03402985]]))

In [101]:
print 'хи-квадрат: {}, \npv: {}, \ndof: {}, \nexpected: \n{}'.format(chi2, pv, dof, expected)

хи-квадрат: 19.4075307885, 
pv: 1.05589870066e-05, 
dof: 1, 
expected: 
[[ 243.03402985  198.96597015]
 [ 677.96597015  555.03402985]]


то же через массив numpy

In [77]:
data_t3_ = [[203, 239],
            [718, 515]]

In [78]:
chi2_contingency(data_t3_)

(19.407530788543038,
 1.0558987006638725e-05,
 1L,
 array([[ 243.03402985,  198.96597015],
        [ 677.96597015,  555.03402985]]))

### сравним с предыдущей формулой для Мэтьюса

In [88]:
corr_Matt_chi2 = np.sqrt(chi2_contingency(data_t3_)[0] / np.sum(data_t3_))
corr_Matt_chi2

0.10764100308617915

### 6

 Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [27]:
import scipy

In [37]:
n1 = 239 + 515 # male
n2 = 203 + 718 # female

p1 = float(239) / n1
p2 = float(203) / n2

In [36]:
def proportions_confint_diff_ind(p1, p2, n1, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
       
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [39]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_ind(p1, p2, n1, n2)

confidence interval: [0.053905, 0.139222]


## 7

In [42]:
def proportions_diff_z_stat_ind(p1, p2, n1, n2):
    
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [43]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [106]:
print 'p-value = {}'.format(proportions_diff_z_test(proportions_diff_z_stat_ind(p1, p2, n1, n2)))

p-value = 8.15345308958e-06


## 8, 9

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

|Не доволен |	Более или менее |	Доволен |
------------- | -------------| -------------|
Не очень счастлив |	197 |	111 |	33 
Достаточно счастлив |	382 |	685 |	331
Очень счастлив |	110 |	342 |	333

8) Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

9) На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [46]:
import scipy

In [104]:
table = [[197, 111, 33], 
         [382, 685, 331],
         [110, 342, 333]]

chi2, pv, dof, expected = scipy.stats.chi2_contingency(table)

In [105]:
print ' chi2 = {}\n pv = {}'.format(chi2, pv)

 chi2 = 293.683110397
 pv = 2.49642995801e-62


## 10
Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

V Cramer == phi_c

In [74]:
n = np.sum(table)
phi_c = np.sqrt(chi2 /(n * (3 - 1)))
phi_c

0.2412013934500338